In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import pkgutil
import importlib
import tqdm

In [ ]:
import evoapproxlib as eal
bitwidth = 12
multipliers = [m.name for m in pkgutil.iter_modules(eal.__path__) if f'mul{bitwidth}s_' in m.name]
attrs = ['MAE_PERCENT', 'MAE', 'WCE_PERCENT', 'WCE', 'WCRE_PERCENT', 'EP_PERCENT', 'MRE_PERCENT', 'MSE', 'PDK45_PWR', 'PDK45_AREA', 'PDK45_DELAY']
for m in multipliers:
    print(m)

In [ ]:
def multiplier_output(multiplier, bitwidth, x, y, signed=True):
    axmul = np.vectorize(multiplier.calc)
    z = axmul(x, y)
    if signed:
        z[z >= 2**(2*bitwidth-1)] -= 2**(2*bitwidth)
    return z

In [ ]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

def sampled_features(bitwidth, num_samples=int(1e6)):
    rng = np.random.default_rng(42)
    x = np.sort(rng.integers(low=-2**(bitwidth-1), high=2**(bitwidth-1), size=num_samples))
    y = np.sort(rng.integers(low=-2**(bitwidth-1), high=2**(bitwidth-1), size=num_samples))
    return np.vstack([x, y]).T

def base_features(bitwidth):
    x = np.arange(-2**(bitwidth-1), 2**(bitwidth-1))
    xx, yy = np.meshgrid(x, x, indexing='ij')
    X = np.vstack([xx.flatten(), yy.flatten()]).T
    return X

def build_model(emap, expansions=[], prune=False, X=None):
    def mod_features(X):
        return np.hstack([X] + [X % sc for sc in expansions])

    # Fit over entire output space
    if X is None:
        X = sampled_features(bitwidth=bitwidth)

    # Build feature transformations
    # Add Modulo features
    features = mod_features(X)
    # Polynomial Expansion
    poly = PolynomialFeatures(degree=2, interaction_only=False)
    features = poly.fit_transform(features)

    # Build Model
    reg = linear_model.LinearRegression()
    reg = reg.fit(features, emap)
    
    # Drop small coefficients
    if prune:
        reg.coef_ = reg.coef_.round(decimals=1)

    class Predictor():
        def __init__(self, mod, poly, regressor):
            self.mod_transform = mod
            self.poly_transform = poly
            self.regressor = regressor

        def __call__(self, X):
            features = self.mod_transform(X)
            features = self.poly_transform.fit_transform(features)
            return self.regressor.predict(features)

    return Predictor(mod_features, poly, reg)

In [ ]:
def wce(emap, model_output):
    ans = np.abs(emap-model_output)
    return np.max(ans)

def mae(emap, model_output):
    ans = np.abs(emap-model_output)
    return np.mean(ans)

def mre(emap, model_output):
    ans = np.abs(emap-model_output)
    div = np.maximum(np.ones_like(ans), ans)
    return np.mean(ans/div)

In [ ]:
def metrics(emap, model, X):
    model_output = np.round(model(X).reshape(emap.shape))
    return {
        "wce" : wce(emap, model_output),
        "mre" : mre(emap, model_output),
        "mae" : mae(emap, model_output),
    }

results = []
if bitwidth == 8:
    bf = base_features()
else:
    bf = sampled_features(bitwidth)

expansions = [float(2**exp) for exp in range(1,10)]

for mname in tqdm.tqdm(multipliers):
    multiplier = importlib.import_module(f"evoapproxlib.{mname}")
    emap = multiplier_output(multiplier, bitwidth, bf[:,0], bf[:,1])
    evoapprox_metrics = dict([(a.lower(), getattr(multiplier, a)) for a in attrs])

    base = build_model(emap, X=bf)
    base_metrics = metrics(emap, base, bf)

    expansion = 0
    best_mae = base_metrics['mae']
    # Grid Search for j=k
    for e in expansions:
        ours = build_model(emap, expansions=[e], prune=True, X=bf)
        cur_mae = metrics(emap, ours, bf)['mae']
        print(cur_mae, best_mae)
        if cur_mae < best_mae:
            expansion = e
            best_mae = cur_mae

    ours = build_model(emap, expansions=[expansion], prune=True, X=bf)
    htp_metrics = metrics(emap, ours, bf)
    mask = ~np.isclose(ours.regressor.coef_, 0)
    htp_params = {
        'coefficients': list(ours.regressor.coef_[mask].tolist()),
        'powers': list(ours.poly_transform.powers_[mask].tolist()),
        'expansion' : expansion,
    }

    results.append({
        'name' : mname,
        'evoapprox_metrics' : evoapprox_metrics,
        'htp_params' : htp_params,
        'baseline_metrics' : base_metrics,
        'htp_metrics' : htp_metrics,
    })

In [ ]:
for r in results:
    print(r['name'], r['baseline_metrics'], r['htp_metrics'])

In [ ]:
import json
with open(f'mul{bitwidth}.json', 'w+') as f:
    json.dump(results, f, indent=4)

In [ ]:
import torchapprox.operators.htp_models.htp_models_mul8s as htp
import torchapprox.utils.evoapprox as evoutil
import torch
import numpy as np

In [ ]:
htp['accurate']

In [ ]:
torch.mul(torch.tensor([10]), torch.tensor([20]))

In [ ]:
lut = evoutil.lut('mul8s_1L12', bitwidth=8, signed=True)

In [ ]:
np.all(lut == lut.T)

In [ ]:
lut

In [ ]:
x = np.arange(-128, 128)
xx, yy = np.meshgrid(x, x)

mname = 'mul8s_1L12'
mul = importlib.import_module(f"evoapproxlib.{mname}")
res = multiplier_output(mul, xx, yy)

In [ ]:
plt.imshow(res, cmap='RdBu')